In [1]:
# !pip install -U  openai datasets -qq

### [Chain-of-Thought Prompting Elicits Reasoning in Large Language Models](https://arxiv.org/pdf/2201.11903)

> Abstract: We explore how generating a chain of thought—a series of **intermediate reasoning**
steps—significantly **improves** the ability of large language models to perform
complex reasoning.

<img src="images/cot.png" >

# Step 1: Importing OpenAI and Writing Our Completion Function

In [1]:
import os
from openai import AzureOpenAI
from llm_config import llm_config, client

def get_completion(messages, model="gpt-35-turbo"):
    completion = client.chat.completions.create(
        model=model,
        messages=messages
        )

    return completion.choices[0].message.content

# Step 2: Loading the MMLU Dataset

In this mini tutorial we are working with MMLU dataset that is used to measure models mutli task accuracy. 


<img src="images/mmlu.png" >


the dataset is made of several task to check if the model have certain abilities 

```
A complete list of tasks: ['abstract_algebra', 'anatomy', 'astronomy', 'business_ethics', 'clinical_knowledge', 'college_biology', 'college_chemistry', 'college_computer_science', 'college_mathematics', 'college_medicine', 'college_physics', 'computer_security', 'conceptual_physics', 'econometrics', 'electrical_engineering', 'elementary_mathematics', 'formal_logic', 'global_facts', 'high_school_biology', 'high_school_chemistry', 'high_school_computer_science', 'high_school_european_history', 'high_school_geography', 'high_school_government_and_politics', 'high_school_macroeconomics', 'high_school_mathematics', 'high_school_microeconomics', 'high_school_physics', 'high_school_psychology', 'high_school_statistics', 'high_school_us_history', 'high_school_world_history', 'human_aging', 'human_sexuality', 'international_law', 'jurisprudence', 'logical_fallacies', 'machine_learning', 'management', 'marketing', 'medical_genetics', 'miscellaneous', 'moral_disputes', 'moral_scenarios', 'nutrition', 'philosophy', 'prehistory', 'professional_accounting', 'professional_law', 'professional_medicine', 'professional_psychology', 'public_relations', 'security_studies', 'sociology', 'us_foreign_policy', 'virology', 'world_religions']
```

We begin by loading the MMLU dataset, specifically focusing on the subject of 'abstract algebra' for our exercises.

In [2]:
from datasets import load_dataset

dataset = load_dataset("cais/mmlu","abstract_algebra")

/Users/israela/Documents/repos/NHR/condor_analysis/.conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We can examine the structure of a single question, along with its choices and answer pair.

In [3]:
Q1 = dataset['test'][0]
Q1

{'question': 'Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.',
 'subject': 'abstract_algebra',
 'choices': ['0', '4', '2', '6'],
 'answer': 1}

# Step 3: Improving Large Language Models with Chain-of-Thought Reasoning


Most advanced LLMs use chain-of-thought (CoT) reasoning for backend processing. To explore the model's responses without this feature, we will explicitly instruct it to provide answers without explanations. This approach will disable CoT reasoning and help us understand how the models perform without it.

In [6]:
messages=[
            {"role": "system", "content": """you are LLM capable of solving algebra questions given a question \
                                        answer without providing any explanation
                                        """},
            {"role": "user", "content": f"Question: {Q1['question']}"}
        ]
response1 = get_completion(messages)


In [7]:
print(response1)

Degree: 8


Now, we can see that the answer quality improves when we specifically request explanations, compared to the gold label we have.

In [8]:
messages=[
            {"role": "system", "content": "you are LLM capable of solving algebra questions given the following \
            question answer thinking step by step"},
            {"role": "user", "content": f"Question: {Q1['question']}"}
        ]
response2 = get_completion(messages)

In [9]:
print(response2)

Step 1: Simplify sqrt(18) by factoring out the perfect square: sqrt(18) = sqrt(9 x 2) = 3sqrt(2)

Step 2: Simplify the field extension by removing duplicates and reducing to a minimal set of generators: Q(sqrt(2), sqrt(3), sqrt(18)) = Q(sqrt(2), sqrt(3))

Step 3: Determine the degree of the extension by finding the product of the degrees of the primitive extensions: [Q(sqrt(2), sqrt(3)) : Q] = [Q(sqrt(2)) : Q][Q(sqrt(3)) : Q]

Step 4: Calculate the degrees of the primitive extensions using the fact that [Q(alpha) : Q] = degree of the minimal polynomial of alpha over Q.

The minimal polynomial of sqrt(2) over Q is x^2 - 2, which has degree 2. Similarly, the minimal polynomial of sqrt(3) over Q is x^2 - 3, which also has degree 2.

Therefore, [Q(sqrt(2), sqrt(3)) : Q] = [Q(sqrt(2)) : Q][Q(sqrt(3)) : Q] = 2 x 2 = 4.

So the degree of the field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q is 4.


# Step 4: Additional Chain-of-Thought Examples

LLMs are easily confused by ambiguous questions, so providing examples can help clarify. Let's see how an LLM might fail on some examples.


In [18]:
messages=[
            {"role": "system", "content": "you are LLM capable answering questions in english give direct short response"},
            {"role": "user", "content": "Take the last letter of the words in 'Lady Gaga' and concatenate them"}
        ]
response3 = get_completion(messages)

In [19]:
response3

'aaga'

Examples always help clarify how LLMs can be confused by certain types of questions.

In [20]:
messages=[
            {"role": "system", "content": "you are LLM capable answering questions in english"},
            {"role": "user", "content": """
             For example if you are given 'Albert Einstein' the last letter of the first word is 't'
             The last letter of the second word is 'n'
             the output should be 'tn'
             
             Question: Take the last letter of the words in 'Lady Gaga' and concatenate them """}
        ]
response4 = get_completion(messages)

In [21]:
response4

"The last letter of the first word is 'y' and the last letter of the second word is 'a'. Therefore, the output will be 'ya'."

## Excercise 1

In [23]:
messages=[
            {"role": "system", "content": "you are LLM capable answering questions in english give direct short response"},
            {"role": "user", "content": "how many t's is in Teletubbies"}
        ]
response5 = get_completion(messages)
response5

"There are three t's in Teletubbies."

In [ ]:
#TODO

# Step 4: Chain-of-Thought and Detailed Output for Error Analysis

when LLMs provide a wrong answer of a complicated quesion its hard to tell on which part that mistake was introuded at. 
Now we will see how we can get insight into the process of thinking and know where they are failing

In [4]:
messages=[
            {"role": "system", "content": "you are LLM capable of solving algebra questions by writing a code,\
              write a python code to solve this problem"},
            {"role": "user", "content": f"Question: {Q1['question']}"}
        ]
response3 = get_completion(messages)

In [5]:
print(response3)

We know that the degree of a field extension is equal to the product of the degrees of the irreducible polynomials which generate the extension. 

First, we observe that:
- sqrt(2) is a root of the irreducible polynomial x^2 - 2 over Q
- sqrt(3) is a root of the irreducible polynomial x^2 -3 over Q
- sqrt(18) is a root of the irreducible polynomial x^2 -18 = (x-sqrt(2)*sqrt(9))(x+sqrt(2)*sqrt(9)) over Q (notice that sqrt(9)=3 is already a term in this polynomial).

Using the above observations, we can now determine the irreducible polynomial of the entire field extension:
- We know Q(sqrt(2)) has degree 2 over Q
- We know Q(sqrt(3), sqrt(2)) = Q(sqrt(3)+sqrt(2), sqrt(3)-sqrt(2)) has degree 2 over Q(sqrt(2)) since those two quantities are roots of x^2 - (2sqrt(2)+3) over Q(sqrt(2)) (we can obtain this by squaring (a+b*sqrt(2)) + (c+d*sqrt(2)), rationalizing denominators and equating coefficients).
- We know Q(sqrt(2), sqrt(3), sqrt(18)) = Q(sqrt(2), sqrt(3)+sqrt(2), sqrt(3)-sqrt(2)) has

In [12]:
print("Degree of Q(sqrt(2), sqrt(3), sqrt(18)) over Q:", deg3)

NameError: name 'deg3' is not defined

In [16]:
messages=[
            {"role": "system", "content": """
                   you are LLM capable of solving algebra questions by writing a code,
                   write a python code to colve this problem.
             
                given a question 
             
                # step 1:
                     extract all the given variables and their value and required value 
             
                # step 2:
                     write a formula that you are going to use to solve the problem
             
                # step 3:
                    write the steps to install all the necessery library
             
                # step 4:
                    write a complete python function that solves a problem
             
                # step 5:
                    Answer :  

             """},
            {"role": "user", "content": f"Question: {Q1['question']}"}
        ]
response4 = get_completion(messages)

In [17]:
print(response4)

# Step 1: 
The given variables and their values are:
- base field: Q (rational numbers)
- extension field: Q(sqrt(2), sqrt(3), sqrt(18))
- square root of 2: sqrt(2)
- square root of 3: sqrt(3)
- required value: degree of field extension

# Step 2: 
By using the multiplicativity property of field degree, we can find the degree of this field extension as the product of degrees of smaller sub-extensions:
degree(Q(sqrt(2), sqrt(3), sqrt(18)) / Q) = degree(Q(sqrt(2), sqrt(3), sqrt(18)) / Q(sqrt(2), sqrt(3))) * degree(Q(sqrt(2), sqrt(3)) / Q(sqrt(2))) * degree(Q(sqrt(2)) / Q)

Now, we need to find each of these degrees.

# Step 3: 
No external libraries are required for this problem.

# Step 4: 
Here is the python code to solve this problem:

def degree_of_extension_field():
    # sub-extension 1: Q(sqrt(2), sqrt(3)) / Q(sqrt(2))
    degree1 = 2   # degree of Q(sqrt(2)) / Q is 2
    degree2 = 2   # degree of Q(sqrt(2), sqrt(3)) / Q(sqrt(2)) is 2, as sqrt(3) is a root of x^2 - 3
    degree3 =

In [38]:
def find_index_of_permutation(p):
    n = len(p)
    index = 0
    for i in range(n-1):
        sigma_i = sum(1 for j in range(i+1, n) if p[j] < p[i])
        factorial = 1
        for k in range(n-i, 0, -1):
            factorial *= k
        index += sigma_i * factorial
    return index

8

# Exercise

Let's apply the same process to the random question below.

In [42]:
Q1 = dataset['test'][1]
Q1

{'question': 'Let p = (1, 2, 5, 4)(2, 3) in S_5 . Find the index of <p> in S_5.',
 'subject': 'abstract_algebra',
 'choices': ['8', '2', '24', '120'],
 'answer': 2}